In [3]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

it took 4.4 hours
'''

import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


In [12]:
##################################################
# Load  
##################################################
input_col = ['msno','w','days_since_the_last_expiration','order_number_rev']
membership_loyalty = utils.read_multiple_csv('../../input/preprocessed_data/days_since_the_last_transactions',input_col) # 20,000,000


100%|██████████| 6/6 [00:00<00:00, 174.20it/s]


# why 有些人這筆訂單的days_since_the_last_subscription == 上一筆訂單的days_since_the_last_expiration,有些人沒有
# 這個人可以選擇跳巢的機會是多少

In [23]:
T  = 0
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','w',]] # we do not need is_churn
membership_loyalty[membership_loyalty.w == T]
df = pd.merge(train, 
        membership_loyalty[membership_loyalty.w == T], 
        on=['msno','w'], 
        how='left')
df = df.dropna() # 這樣會drop掉有些不該drop的欄位


# 考慮此消費者所有的消費記錄

In [24]:
col =['msno','days_since_the_last_expiration-mean', 'days_since_the_last_expiration-std',
     'days_since_the_last_expiration-min','days_since_the_last_expiration-max',
     'days_since_the_last_expiration-50%']
df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration-').reset_index(level=0)[col]

In [25]:
folder = 'trainW-0'
df.to_csv('../../feature/{}/days_since_the_last_expiration.csv'.format(folder), index = False)
df


,msno,days_since_the_last_expiration-mean,days_since_the_last_expiration-std,days_since_the_last_expiration-min,days_since_the_last_expiration-max,days_since_the_last_expiration-50%
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,-74.750000,38.870512,-119.0,-29.0,-75.5
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,-411.950000,263.025209,-809.0,-47.0,-337.5
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,-410.346154,233.203935,-790.0,-31.0,-410.5
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,-198.750000,110.021589,-365.0,-31.0,-199.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,-415.950000,263.025209,-813.0,-51.0,-341.5
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,-411.909091,204.431129,-735.0,-30.0,-415.5
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,-198.750000,110.021589,-365.0,-31.0,-199.0
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,-263.250000,174.809039,-571.0,-30.0,-245.5
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,-150.166667,99.850722,-321.0,-30.0,-129.5
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,-257.812500,145.147382,-485.0,-29.0,-257.5


# near 5

In [22]:
# merge
df = pd.merge(train, 
    membership_loyalty[(membership_loyalty.w == T) & (membership_loyalty.order_number_rev <=5)], 
    on=['msno','w'], 
    how='left')
# core
col =['msno','days_since_the_last_expiration_n5-mean', 'days_since_the_last_expiration_n5-std',
 'days_since_the_last_expiration_n5-min','days_since_the_last_expiration_n5-max',
 'days_since_the_last_expiration_n5-50%']

df = df.dropna()

df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration_n5-').reset_index(level=0)[col]
# write
#df.to_csv('../feature/{}/days_since_the_last_expiration.csv'.format(folder), index = False)
df

,msno,days_since_the_last_expiration_n5-mean,days_since_the_last_expiration_n5-std,days_since_the_last_expiration_n5-min,days_since_the_last_expiration_n5-max,days_since_the_last_expiration_n5-50%
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,-74.75,38.870512,-119.0,-29.0,-75.5
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,-748.80,47.599370,-809.0,-688.0,-750.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,-730.60,47.757722,-790.0,-670.0,-731.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,-305.60,47.757722,-365.0,-245.0,-306.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,-752.80,47.599370,-813.0,-692.0,-754.0
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,-674.80,47.599370,-735.0,-614.0,-676.0
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,-305.60,47.757722,-365.0,-245.0,-306.0
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,-472.40,95.395493,-571.0,-356.0,-508.0
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,-174.20,90.170394,-321.0,-94.0,-133.0
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,-425.60,47.757722,-485.0,-365.0,-426.0


In [28]:
membership_loyalty[(membership_loyalty.w == T) ]

,msno,w,days_since_the_last_expiration,order_number_rev
0,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-29,9.0
1,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-60,8.0
2,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-91,7.0
3,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-121,6.0
4,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-152,5.0
5,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-182,4.0
6,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-213,3.0
7,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-244,2.0
8,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-272,1.0
9,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0,-31,26.0


In [130]:
aList = [123, 'xyz', 'zara', 'abc', 'xyz']
list(reversed(aList))

['xyz', 'abc', 'zara', 'xyz', 123]

In [127]:
aList

['xyz', 'abc', 'zara', 'xyz', 123]

In [30]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory

##################################################
# Load  
##################################################
input_col = ['msno','w','days_since_the_last_expiration','order_number_rev']
membership_loyalty = utils.read_multiple_csv('../../input/preprocessed_data/days_since_the_last_transactions',input_col) # 20,000,000





100%|██████████| 6/6 [00:00<00:00, 215.60it/s]


In [38]:
#==============================================================================
# def
#==============================================================================
def make(T):
	"""
	T = 0
	folder = 'trainW-0'
	"""

	if T==-1:
	    folder = 'test'
	    train = pd.read_csv('../../input/sample_submission_v2.csv') # 此train代表的是test的user
	    train['w'] = -1
	else:
	    folder = 'trainW-'+str(T)
	    train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno','w',]] # we do not need is_churn
	##################################################
	# All history
	##################################################
	# merge
	df = pd.merge(train, 
	    membership_loyalty[membership_loyalty.w == T], 
	    on=['msno','w'], 
	    how='left')
	# core
	col =['msno','days_since_the_last_expiration-mean', 'days_since_the_last_expiration-std',
	 'days_since_the_last_expiration-min','days_since_the_last_expiration-max',
	 'days_since_the_last_expiration-50%']

	df = df.dropna()

	df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration-').reset_index(level=0)[col]
	# write
	df.to_csv('../../feature/{}/days_since_the_last_expiration.csv'.format(folder), index = False)
	##################################################
	# near 5
	##################################################
	# merge
	df = pd.merge(train, 
	    membership_loyalty[(membership_loyalty.w == T) & (membership_loyalty.order_number_rev <=5)], 
	    on=['msno','w'], 
	    how='left')
	# core
	col =['msno','days_since_the_last_expiration_n5-mean', 'days_since_the_last_expiration_n5-std',
	 'days_since_the_last_expiration_n5-min','days_since_the_last_expiration_n5-max',
	 'days_since_the_last_expiration_n5-50%']

	df = df.dropna()

	df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration_n5-').reset_index(level=0)[col]
	# write
	df.to_csv('../../feature/{}/days_since_the_last_expiration_n5.csv'.format(folder), index = False)

	##################################################
	# only one prvious order
	##################################################
	# merge
	df = pd.merge(train, 
	    membership_loyalty[(membership_loyalty.w == T) & (membership_loyalty.order_number_rev == 1)], 
	    on=['msno','w'], 
	    how='left')
	# core
	col =['msno','days_since_the_last_expiration_n5-mean', 'days_since_the_last_expiration_n5-std',
	 'days_since_the_last_expiration_n5-min','days_since_the_last_expiration_n5-max',
	 'days_since_the_last_expiration_n5-50%']

	df = df.dropna()

	df = df.groupby('msno').days_since_the_last_expiration.describe().add_prefix('days_since_the_last_expiration_n5-').reset_index(level=0)[col]
	# write
	df.to_csv('../../feature/{}/days_since_the_last_expiration_n5.csv'.format(folder), index = False)
	del df
	del train
	gc.collect()
	print ('{0} done'.format(T))




In [39]:
##################################################
# Main
##################################################
s = time.time()
make(0)
make(1)
make(2)
make(-1)
e = time.time()
print (e-s)



0 done
1 done
2 done
-1 done
14.808813095092773


In [67]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

it took 4.4 hours
'''

import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


##################################################
# Load  
##################################################
col = ['msno','plan_list_price','payment_plan_days','actual_amount_paid','payment_method_id','transaction_date','membership_expire_date']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions', col) # 20,000,000

#transactions = transactions.head(n = 500)

input_col = ['msno','t-1_transaction_date','t-1_membership_expire_date','t-1_actual_amount_paid',
             'order_number_rev','t-1_payment_method_id','t-1_plan_list_price','t-1_payment_plan_days']
tran_time_diff = utils.read_multiple_csv('../../input/preprocessed_data/transactions_time_diff', input_col)
tran_time_diff = tran_time_diff.dropna() # drop columns whose oder_number == 1

#tran_time_diff = tran_time_diff.head(n = 500)


100%|██████████| 4/4 [00:35<00:00,  8.84s/it]


In [14]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22964064 entries, 0 to 5741015
Data columns (total 7 columns):
msno                      object
plan_list_price           int64
payment_plan_days         int64
actual_amount_paid        int64
payment_method_id         int64
transaction_date          object
membership_expire_date    object
dtypes: int64(4), object(3)
memory usage: 1.4+ GB


In [15]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(transactions)
transactions.info()

reduce memory


100%|██████████| 7/7 [00:00<00:00, 2080.80it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 22964064 entries, 0 to 5741015
Data columns (total 7 columns):
msno                      object
plan_list_price           int16
payment_plan_days         int16
actual_amount_paid        int16
payment_method_id         int8
transaction_date          object
membership_expire_date    object
dtypes: int16(3), int8(1), object(3)
memory usage: 854.1+ MB


In [16]:
tran_time_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20538102 entries, 2 to 5741013
Data columns (total 8 columns):
msno                          object
t-1_transaction_date          object
t-1_membership_expire_date    object
t-1_actual_amount_paid        float64
order_number_rev              int64
t-1_payment_method_id         float64
t-1_plan_list_price           float64
t-1_payment_plan_days         float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.4+ GB


In [17]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tran_time_diff)
tran_time_diff.info()

reduce memory


100%|██████████| 8/8 [00:00<00:00, 1611.33it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20538102 entries, 2 to 5741013
Data columns (total 8 columns):
msno                          object
t-1_transaction_date          object
t-1_membership_expire_date    object
t-1_actual_amount_paid        int16
order_number_rev              int8
t-1_payment_method_id         int8
t-1_plan_list_price           int16
t-1_payment_plan_days         int16
dtypes: int16(3), int8(2), object(3)
memory usage: 783.5+ MB


In [98]:
# transactions = transactions.head(n = 500)
# tran_time_diff = tran_time_diff.head(n = 500)
# ##################################################
# # Convert string to datetime format
# ##################################################
# transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# tran_time_diff['t-1_membership_expire_date']  = tran_time_diff['t-1_membership_expire_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# tran_time_diff['t-1_transaction_date']  = tran_time_diff['t-1_transaction_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

# gc.collect()

print ('After Loading ...')

def days_since_the_last_expiration(x):
    # x: each row of dataframe, series
    x['days_since_the_last_expiration'] = [i.days for i in (x.transaction_date - x['t-1_membership_expire_date'])]
    return x
def days_since_the_last_subscription(x):
    x['days_since_the_last_subscription'] = [i.days for i in (x.transaction_date - x['t-1_transaction_date'])]
    return x
def is_subscribe_early(x):
    x['is_subscribe_early'] = [1 if i <0 else 0 for i in x.days_since_the_last_expiration]
    return x
def creat_loyalty_trend(x):
    # date
    x['days_since_the_last_expiration-cumsum'] = x.days_since_the_last_expiration.cumsum()
    x['days_since_the_last_expiration_ratio'] = x.days_since_the_last_expiration.cumsum()/ list(reversed(x.order_number_rev.tolist()))
    x['days_since_the_last_subscription_ratio'] = x.days_since_the_last_subscription.cumsum()/ list(reversed(x.order_number_rev.tolist()))
    x['days_since_the_last_expiration_diff'] = x.days_since_the_last_expiration - x.days_since_the_last_expiration.shift(1)
    x['days_since_the_first_subscription'] = x.days_since_the_last_subscription.cumsum()
    # payment_method
    x['do_change_payment_method'] = [1 if (p_m - t_1_p_m) != 0 else 0 for p_m, t_1_p_m in x[['payment_method_id','t-1_payment_method_id']].values]
    # plan_list_price(這次訂單,有選擇更高的價錢的方案麼)
    x['do_spend_more_money'] = [p_price - t_1_p_price for p_price, t_1_p_price in x[['plan_list_price','t-1_plan_list_price']].values]
    # payment_plan_days(這次訂單,有選擇天數更高的方案麼)
    x['do_extend_payment_days'] = [p_p_days - t_1_p_days for p_p_days, t_1_p_days in x[['payment_plan_days','t-1_payment_plan_days']].values]
    # (這次訂單,真實付的錢有比上次多麽)
    x['do_paid_more'] = [a_paid - a_paid for a_paid, a_paid in x[['actual_amount_paid','t-1_actual_amount_paid']].values]
    return x.tail(x.shape[0] -1) # 不取最後一筆
days_since_the_last_transactions = []


After Loading ...


In [40]:
#==============================================================================
# def
#==============================================================================
def make(T):
    """
    T = 0
    folder = 'trainW-0'
    """
    if T ==-1:
        folder = 'test'
        test = pd.read_csv('../../input/sample_submission_v2.csv')
    else:
        folder = 'trainW-'+str(T)
        train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))
    #merge data

    if T == 0:
        # w = 0:使用3月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-02-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-01-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        df = pd.merge(test, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-04-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del test
        df['w'] = T 
    gc.collect()

    df = pd.merge(df, tran_time_diff, on='msno', how='left').drop_duplicates(['msno','order_number_rev'])   
    
    df = df.dropna()

    # creating features
    df = df.groupby('msno').apply(days_since_the_last_expiration)
    df = df.groupby('msno').apply(days_since_the_last_subscription)
    df = df.groupby('msno').apply(is_subscribe_early)
    df = df.groupby('msno').apply(creat_loyalty_trend)

    output_col = ['msno','w','days_since_the_last_expiration',
            'days_since_the_last_subscription','is_subscribe_early','order_number_rev',
            'days_since_the_last_expiration-cumsum',
            'days_since_the_last_expiration_ratio',
            'days_since_the_last_subscription_ratio',
            'days_since_the_last_expiration_diff',
            'days_since_the_first_subscription',
                  'do_change_payment_method',
                  'do_spend_more_money',
                  'do_extend_payment_days',
                  'do_paid_more',
            ]
    df = df[output_col] # msno is not unique
    gc.collect()

    days_since_the_last_transactions.append(df)
    print ('{0} done'.format(T))
    del df
    gc.collect()

In [41]:
make(0)
make(1)
make(2)
make(-1)


0 done
1 done
2 done
-1 done


In [24]:
c = pd.concat(days_since_the_last_transactions, ignore_index = True)
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1383 entries, 0 to 1382
Data columns (total 15 columns):
msno                                      1383 non-null object
w                                         1383 non-null int64
days_since_the_last_expiration            1383 non-null int64
days_since_the_last_subscription          1383 non-null int64
is_subscribe_early                        1383 non-null int64
order_number_rev                          1383 non-null float64
days_since_the_last_expiration-cumsum     1383 non-null int64
days_since_the_last_expiration_ratio      1383 non-null float64
days_since_the_last_subscription_ratio    1383 non-null float64
days_since_the_last_expiration_diff       1299 non-null float64
days_since_the_first_subscription         1383 non-null int64
do_change_payment_method                  1383 non-null int64
do_spend_more_money                       1383 non-null float64
do_extend_payment_days                    1383 non-null float64
do_paid_more

In [28]:
transactions[transactions.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']

,msno,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,transaction_date,membership_expire_date
242,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-06-22,2016-07-21
243,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-07-21,2016-08-21
244,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-08-21,2016-09-21
245,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-09-21,2016-10-21
246,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-10-21,2016-11-21
247,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-11-21,2016-12-21
248,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2016-12-21,2017-01-21
249,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2017-01-21,2017-02-21
250,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2017-02-21,2017-03-21
251,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,99,30,99,41,2017-03-21,2017-04-21


In [68]:
T = 0
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))

In [71]:
df = pd.merge(train, 
transactions[(transactions.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on='msno', 
how='left') # 此時msno就是不重複


In [72]:
df[df.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']

,msno,is_churn,w,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,transaction_date,membership_expire_date
58581,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21
58582,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-07-21,2016-08-21
58583,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-08-21,2016-09-21
58584,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-09-21,2016-10-21
58585,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-10-21,2016-11-21
58586,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-11-21,2016-12-21
58587,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-12-21,2017-01-21
58588,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-01-21,2017-02-21
58589,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-02-21,2017-03-21


In [73]:
tran_time_diff[tran_time_diff.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']

,msno,t-1_transaction_date,t-1_membership_expire_date,t-1_actual_amount_paid,order_number_rev,t-1_payment_method_id,t-1_plan_list_price,t-1_payment_plan_days
243,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-06-22,2016-07-21,99.0,9,41.0,99.0,30.0
244,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-07-21,2016-08-21,99.0,8,41.0,99.0,30.0
245,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-08-21,2016-09-21,99.0,7,41.0,99.0,30.0
246,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-09-21,2016-10-21,99.0,6,41.0,99.0,30.0
247,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-10-21,2016-11-21,99.0,5,41.0,99.0,30.0
248,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-11-21,2016-12-21,99.0,4,41.0,99.0,30.0
249,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-12-21,2017-01-21,99.0,3,41.0,99.0,30.0
250,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-01-21,2017-02-21,99.0,2,41.0,99.0,30.0
251,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-02-21,2017-03-21,99.0,1,41.0,99.0,30.0


In [58]:
df = pd.merge(df, tran_time_diff, on='msno', how='left').drop_duplicates(['msno','order_number_rev'])   

#df = df.dropna()


In [59]:
df[df.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']

,msno,is_churn,w,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,transaction_date,membership_expire_date,t-1_transaction_date,t-1_membership_expire_date,t-1_actual_amount_paid,order_number_rev,t-1_payment_method_id,t-1_plan_list_price,t-1_payment_plan_days
0,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-06-22,2016-07-21,99,9,41,99,30
1,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-07-21,2016-08-21,99,8,41,99,30
2,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-08-21,2016-09-21,99,7,41,99,30
3,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-09-21,2016-10-21,99,6,41,99,30
4,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-10-21,2016-11-21,99,5,41,99,30
5,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-11-21,2016-12-21,99,4,41,99,30
6,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2016-12-21,2017-01-21,99,3,41,99,30
7,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2017-01-21,2017-02-21,99,2,41,99,30
8,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21,2017-02-21,2017-03-21,99,1,41,99,30


In [77]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

It took 5.446 hours
'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory

##################################################
# Load transaction 
##################################################
#input_col = []
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions')
#print ('shape of transactions',transactions.shape)
transactions = transactions.head(n = 500)
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))



print ('After Loading ...')
##################################################
# 善用time(時間差)來區隔交易日期時間點的行為
##################################################

def t_diff(x, time_diff):
    # x : the dataframe belong to a each specific user
    # time_diff: 2,3,4,...
    for i in range(1, time_diff):
        x['t-{}_transaction_date'.format(i)] = x['transaction_date'].shift(i)
        x['t-{}_membership_expire_date'.format(i)] = x['membership_expire_date'].shift(i)
        x['t-{}_payment_method_id'.format(i)] = x['payment_method_id'].shift(i)
        x['t-{}_payment_plan_days'.format(i)] = x['payment_plan_days'].shift(i)
        x['t-{}_plan_list_price'.format(i)] = x['plan_list_price'].shift(i)
        x['t-{}_actual_amount_paid'.format(i)] = x['actual_amount_paid'].shift(i)
        x['t-{}_is_auto_renew'.format(i)] = x['is_auto_renew'].shift(i)
        x['t-{}_is_cancel'.format(i)] = x['is_cancel'].shift(i)
    return x
def make_order_number(x):
    # for near 5
    # 1:代表最靠近現在,數字越大代表是越過去的歷史紀錄
    x['order_number_rev'] = list(reversed([i+1 for i in range(x.shape[0])]))
    return x

##################################################
# Main
##################################################
s = time.time()

transactions = transactions.groupby('msno').apply(t_diff, 2)
transactions = transactions.groupby('msno').apply(make_order_number)

gc.collect()


100%|██████████| 4/4 [00:26<00:00,  6.58s/it]


After Loading ...


33

In [82]:
output_col = ['msno','transaction_date','t-1_transaction_date','t-1_membership_expire_date',
              't-1_payment_method_id','t-1_payment_plan_days',
             't-1_plan_list_price','t-1_actual_amount_paid',
             't-1_is_auto_renew','t-1_is_cancel',
             'order_number_rev'
             ]

c = transactions[transactions.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro='][output_col]
c

,msno,transaction_date,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev
242,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-06-22,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,10
243,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-07-21,2016-06-22,2016-07-21,41.0,30.0,99.0,99.0,1.0,0.0,9
244,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-08-21,2016-07-21,2016-08-21,41.0,30.0,99.0,99.0,1.0,0.0,8
245,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-09-21,2016-08-21,2016-09-21,41.0,30.0,99.0,99.0,1.0,0.0,7
246,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-10-21,2016-09-21,2016-10-21,41.0,30.0,99.0,99.0,1.0,0.0,6
247,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-11-21,2016-10-21,2016-11-21,41.0,30.0,99.0,99.0,1.0,0.0,5
248,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-12-21,2016-11-21,2016-12-21,41.0,30.0,99.0,99.0,1.0,0.0,4
249,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-01-21,2016-12-21,2017-01-21,41.0,30.0,99.0,99.0,1.0,0.0,3
250,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-02-21,2017-01-21,2017-02-21,41.0,30.0,99.0,99.0,1.0,0.0,2
251,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-03-21,2017-02-21,2017-03-21,41.0,30.0,99.0,99.0,1.0,0.0,1


In [83]:
d = df[df.msno == '++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=']
d

,msno,is_churn,w,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,transaction_date,membership_expire_date
58581,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-06-22,2016-07-21
58582,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-07-21,2016-08-21
58583,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-08-21,2016-09-21
58584,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-09-21,2016-10-21
58585,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-10-21,2016-11-21
58586,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-11-21,2016-12-21
58587,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2016-12-21,2017-01-21
58588,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-01-21,2017-02-21
58589,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,0,99.0,30.0,99.0,41.0,2017-02-21,2017-03-21


In [99]:
qqqq = pd.merge(c,d , on =['msno','transaction_date'])
qqqq

,msno,transaction_date,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev,is_churn,w,plan_list_price,payment_plan_days,actual_amount_paid,payment_method_id,membership_expire_date
0,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-06-22,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,10,0,0,99.0,30.0,99.0,41.0,2016-07-21
1,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-07-21,2016-06-22,2016-07-21,41.0,30.0,99.0,99.0,1.0,0.0,9,0,0,99.0,30.0,99.0,41.0,2016-08-21
2,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-08-21,2016-07-21,2016-08-21,41.0,30.0,99.0,99.0,1.0,0.0,8,0,0,99.0,30.0,99.0,41.0,2016-09-21
3,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-09-21,2016-08-21,2016-09-21,41.0,30.0,99.0,99.0,1.0,0.0,7,0,0,99.0,30.0,99.0,41.0,2016-10-21
4,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-10-21,2016-09-21,2016-10-21,41.0,30.0,99.0,99.0,1.0,0.0,6,0,0,99.0,30.0,99.0,41.0,2016-11-21
5,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-11-21,2016-10-21,2016-11-21,41.0,30.0,99.0,99.0,1.0,0.0,5,0,0,99.0,30.0,99.0,41.0,2016-12-21
6,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-12-21,2016-11-21,2016-12-21,41.0,30.0,99.0,99.0,1.0,0.0,4,0,0,99.0,30.0,99.0,41.0,2017-01-21
7,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-01-21,2016-12-21,2017-01-21,41.0,30.0,99.0,99.0,1.0,0.0,3,0,0,99.0,30.0,99.0,41.0,2017-02-21
8,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-02-21,2017-01-21,2017-02-21,41.0,30.0,99.0,99.0,1.0,0.0,2,0,0,99.0,30.0,99.0,41.0,2017-03-21


In [100]:
qqqq = qqqq.groupby('msno').apply(days_since_the_last_expiration)
qqqq = qqqq.groupby('msno').apply(days_since_the_last_subscription)
qqqq = qqqq.groupby('msno').apply(is_subscribe_early)
qqqq = qqqq.groupby('msno').apply(creat_loyalty_trend)


In [103]:
output_col = ['msno','w','transaction_date','t-1_transaction_date',
              'membership_expire_date','t-1_membership_expire_date',
              'days_since_the_last_expiration',
            'days_since_the_last_subscription','is_subscribe_early','order_number_rev',
            'days_since_the_last_expiration-cumsum',
            'days_since_the_last_expiration_ratio',
            'days_since_the_last_subscription_ratio',
            'days_since_the_last_expiration_diff',
            'days_since_the_first_subscription',
                  'do_change_payment_method',
                  'do_spend_more_money',
                  'do_extend_payment_days',
                  'do_paid_more',
            ]
qqqq = qqqq[output_col]
qqqq.reset_index(drop = True)

,msno,w,transaction_date,t-1_transaction_date,membership_expire_date,t-1_membership_expire_date,days_since_the_last_expiration,days_since_the_last_subscription,is_subscribe_early,order_number_rev,days_since_the_last_expiration-cumsum,days_since_the_last_expiration_ratio,days_since_the_last_subscription_ratio,days_since_the_last_expiration_diff,days_since_the_first_subscription,do_change_payment_method,do_spend_more_money,do_extend_payment_days,do_paid_more
0,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2016-07-21,2016-06-22,2016-08-21,2016-07-21,0.0,29.0,0,9,0.0,0.0,9.666667,NaN,29.0,0,0.0,0.0,0.0
1,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2016-08-21,2016-07-21,2016-09-21,2016-08-21,0.0,31.0,0,8,0.0,0.0,15.000000,0.0,60.0,0,0.0,0.0,0.0
2,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2016-09-21,2016-08-21,2016-10-21,2016-09-21,0.0,31.0,0,7,0.0,0.0,18.200000,0.0,91.0,0,0.0,0.0,0.0
3,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2016-10-21,2016-09-21,2016-11-21,2016-10-21,0.0,30.0,0,6,0.0,0.0,20.166667,0.0,121.0,0,0.0,0.0,0.0
4,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2016-11-21,2016-10-21,2016-12-21,2016-11-21,0.0,31.0,0,5,0.0,0.0,21.714286,0.0,152.0,0,0.0,0.0,0.0
5,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2016-12-21,2016-11-21,2017-01-21,2016-12-21,0.0,30.0,0,4,0.0,0.0,22.750000,0.0,182.0,0,0.0,0.0,0.0
6,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2017-01-21,2016-12-21,2017-02-21,2017-01-21,0.0,31.0,0,3,0.0,0.0,23.666667,0.0,213.0,0,0.0,0.0,0.0
7,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,2017-02-21,2017-01-21,2017-03-21,2017-02-21,0.0,31.0,0,2,0.0,0.0,24.400000,0.0,244.0,0,0.0,0.0,0.0
